In [1]:
import scipy.io
import os
from skimage import io, color, transform, exposure
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
temp = scipy.io.loadmat('../../data/OxfordFlowers/imagelabels.mat')
labels = temp['labels']
labels = labels.flatten()

In [3]:
labels.shape

(8189,)

In [4]:
# temp = scipy.io.loadmat('../../data/OxfordFlowers/setid.mat')
# trnid = temp['trnid'].flatten()
# valid = temp['valid'].flatten()
# tstid = temp['tstid'].flatten()

# print(trnid.shape, valid.shape, tstid.shape)

In [5]:
imagepath = '../../data/OxfordFlowers/jpg'
segpath = '../../data/OxfordFlowers/segmim'

filenames = os.listdir(imagepath)
imagefiles = []
segmfiles = []
for i in range(len(filenames)):
    imagefiles.append(imagepath + '/' + filenames[i])
    segmfiles.append(segpath + '/' + filenames[i])

data = {'imagefiles':imagefiles, 'segmfiles':segmfiles, 'labels':labels}
alldata = pd.DataFrame(data)
alldata['labels'] = alldata.labels.astype(str)
alldata.head()

,imagefiles,segmfiles,labels
0,../../data/OxfordFlowers/jpg/image_07088.jpg,../../data/OxfordFlowers/segmim/image_07088.jpg,77
1,../../data/OxfordFlowers/jpg/image_06396.jpg,../../data/OxfordFlowers/segmim/image_06396.jpg,77
2,../../data/OxfordFlowers/jpg/image_05847.jpg,../../data/OxfordFlowers/segmim/image_05847.jpg,77
3,../../data/OxfordFlowers/jpg/image_04581.jpg,../../data/OxfordFlowers/segmim/image_04581.jpg,77
4,../../data/OxfordFlowers/jpg/image_03588.jpg,../../data/OxfordFlowers/segmim/image_03588.jpg,77


In [6]:
from random import shuffle
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from sklearn.model_selection import train_test_split



In [7]:
INPUT_SIZE = 500
BATCH_SIZE = 16
NUMCLASSES = len(alldata.labels.unique())

In [8]:
def preprocess(img):
    if len(img.shape)==2:
        img = np.stack([img, img, img], axis = 2)

    img = img - np.min(img)
    img = img / np.max(img)
    
    img = img - np.mean(img)
    img = img / np.std(img)   
    return img


In [9]:
traindata, testdata = train_test_split(alldata, test_size=0.2,
                                                                stratify=alldata.labels)
print(traindata.shape,testdata.shape)
traindata = traindata.sample(frac=1.0)

(6551, 3) (1638, 3)


In [10]:
train_datagen = ImageDataGenerator(
                                featurewise_center=False,
                                samplewise_center=False,
                                featurewise_std_normalization=False,
                                samplewise_std_normalization=False,
                                zca_epsilon=1e-06,
                                rotation_range=5,
                                width_shift_range=0.05,
                                height_shift_range=0.05,
                                brightness_range=None,
                                shear_range=0.5,
                                zoom_range=0.05,
                                channel_shift_range=0.0,
                                fill_mode="nearest",
                                cval=0.0,
                                horizontal_flip=True,
                                vertical_flip=False,
                                rescale=None,
                                preprocessing_function=preprocess,
                                data_format=None,
                                validation_split=0.2,
                                dtype=None,
                            )


In [12]:
training_set = train_datagen.flow_from_dataframe(
                                            dataframe=traindata,
                                            directory='',
                                            x_col="imagefiles",
                                            y_col="labels",
                                            #weight_col=None,
                                            target_size=(INPUT_SIZE, INPUT_SIZE),
                                            color_mode="rgb",
                                            classes=None,
                                            class_mode="categorical",
                                            batch_size=BATCH_SIZE,
                                            shuffle=True,
                                            seed=None,
                                            save_to_dir=None,
                                            save_prefix="",
                                            save_format="png",
                                            subset='training',
                                            interpolation="nearest",
                                            #validate_filenames=True
                                        )

validation_set = train_datagen.flow_from_dataframe( dataframe=traindata,
                                            directory='',
                                            x_col="imagefiles",
                                            y_col="labels",
                                            #weight_col=None,
                                            target_size=(INPUT_SIZE, INPUT_SIZE),
                                            color_mode="rgb",
                                            classes=None,
                                            class_mode="categorical",
                                            batch_size=BATCH_SIZE,
                                            shuffle=True,
                                            seed=None,
                                            save_to_dir=None,
                                            save_prefix="",
                                            save_format="png",
                                            subset='validation',
                                            interpolation="nearest",
                                            #validate_filenames=True
                                            )

Found 5241 validated image filenames belonging to 102 classes.
Found 1310 validated image filenames belonging to 102 classes.


In [13]:
model = Sequential()

model.add(InputLayer(input_shape=[INPUT_SIZE,INPUT_SIZE,3])) #keras will internally add batch dimension

model.add(Conv2D(filters=64,kernel_size=7,strides=3,padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=3,padding='same'))
model.add(Dropout(0.4))

model.add(Conv2D(filters=64,kernel_size=5,strides=2,padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2,padding='same'))
model.add(Dropout(0.4))

model.add(Conv2D(filters=128,kernel_size=3,strides=1,padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2,padding='same'))
model.add(Dropout(0.4))
model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(NUMCLASSES,activation='softmax'))

model.add(Dense(64,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(NUMCLASSES,activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 167, 167, 64)      9472      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 56, 56, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 56, 56, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        102464    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 14, 14, 64)        0

In [ ]:
#Training

epochs = 10
history = model.fit(training_set,
               epochs=epochs,
               validation_data=validation_set)

model.save_weights("flowers-model-10.weights.h5")
print("Saved model to disk after",epochs,"epochs.")


Epoch 1/10


2025-04-21 14:05:30.762146: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


123/328 [==========>...................] - ETA: 1:01 - loss: 4.6005 - accuracy: 0.0204    